In [1]:
rm(list=ls())


# Load libraries
library(randomForest)
library(DataExplorer)
library(forecast)
library(tidyverse)
library(tseries)
library(ggplot2)

# Read in data
train <- read.csv("../input/sales-test-and-train/train.csv")
test <- read.csv("../input/sales-test-and-train/test.csv")
fulldata <- bind_rows(train, test)

# Do some transformations
fulldata$date <- as.Date(fulldata$date)
fulldata <- fulldata %>% mutate_at(vars(store, item), factor)

# Break into train and test again
train <- fulldata %>% filter(is.na(sales) == FALSE)
test <- fulldata %>% filter(is.na(sales) == TRUE)

# Initialize dataframe to save all predictions
preds <- data.frame()

# Create nested for loop to subset, train, and predict
for (s in 1:10) {
  for (i in 1:50) {
    # Subset
    temp_train <- train %>% filter((store == s) & (item == i)) %>% select(date, sales)
    temp_test <- test %>% filter((store == s) & (item == i))
    
    ## Prepare to fit ARIMA model
    ts_train <- ts(temp_train[,2], start=c(2013,1), frequency = 365)
    # Decomposing data
    #decomp <- stl(ts_train, s.window="periodic")
    #deseasonal <- seasadj(decomp) # Remove seasonality
    # Fit model
    fit <- auto.arima(ts_train, seasonal=TRUE)#, max.p = 15)
    # Predict
    seas_fcast <- forecast(fit, h=90)
    
    # Join predictions to subset of test data
    temp_test$sales <- as.vector(seas_fcast$mean)
    
    # Save predictions
    preds <- rbind(preds, temp_test)
    
    # Tracking
    print(paste("Store number:", s, "Item number:", i))
  }
}



randomForest 4.6-14

Type rfNews() to see new features/changes/bug fixes.

Registered S3 method overwritten by 'quantmod':
  method            from
  as.zoo.data.frame zoo 

── Attaching packages ─────────────────────────────────────── tidyverse 1.3.0 ──

✔ ggplot2 3.3.2     ✔ purrr   0.3.4
✔ tibble  3.0.4     ✔ dplyr   1.0.2
✔ tidyr   1.1.2     ✔ stringr 1.4.0
✔ readr   1.4.0     ✔ forcats 0.5.0

── Conflicts ────────────────────────────────────────── tidyverse_conflicts() ──
✖ dplyr::combine()  masks randomForest::combine()
✖ dplyr::filter()   masks stats::filter()
✖ dplyr::lag()      masks stats::lag()
✖ ggplot2::margin() masks randomForest::margin()



[1] "Store number: 1 Item number: 1"
[1] "Store number: 1 Item number: 2"
[1] "Store number: 1 Item number: 3"
[1] "Store number: 1 Item number: 4"
[1] "Store number: 1 Item number: 5"
[1] "Store number: 1 Item number: 6"
[1] "Store number: 1 Item number: 7"
[1] "Store number: 1 Item number: 8"
[1] "Store number: 1 Item number: 9"
[1] "Store number: 1 Item number: 10"
[1] "Store number: 1 Item number: 11"
[1] "Store number: 1 Item number: 12"
[1] "Store number: 1 Item number: 13"
[1] "Store number: 1 Item number: 14"
[1] "Store number: 1 Item number: 15"
[1] "Store number: 1 Item number: 16"
[1] "Store number: 1 Item number: 17"
[1] "Store number: 1 Item number: 18"
[1] "Store number: 1 Item number: 19"
[1] "Store number: 1 Item number: 20"
[1] "Store number: 1 Item number: 21"
[1] "Store number: 1 Item number: 22"
[1] "Store number: 1 Item number: 23"
[1] "Store number: 1 Item number: 24"
[1] "Store number: 1 Item number: 25"
[1] "Store number: 1 Item number: 26"
[1] "Store number: 1 

In [2]:
dim(preds)
head(preds)

preds_final <- preds %>% select(id, sales)
head(preds_final)
write_csv(preds_final, 'submission.csv', col_names = TRUE)

[1] 45000     5

,date,store,item,sales,id
,<date>,<fct>,<fct>,<dbl>,<int>
1,2018-01-01,1,1,16.04824,0
2,2018-01-02,1,1,15.09720,1
3,2018-01-03,1,1,15.86333,2
4,2018-01-04,1,1,16.23392,3
5,2018-01-05,1,1,18.04687,4
6,2018-01-06,1,1,19.15379,5


,id,sales
,<int>,<dbl>
1,0,16.04824
2,1,15.09720
3,2,15.86333
4,3,16.23392
5,4,18.04687
6,5,19.15379
